#  cropping and saving after deteching by yolo

In [17]:
import cv2 
import argparse
import numpy as np
import os.path
from matplotlib import pyplot as plt
%matplotlib inline

def imshow(tit, image) :
    plt.title(tit)    
    if len(image.shape) == 3 :
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    else :
        plt.imshow(image, cmap="gray")
    plt.show()

## yolo model load

In [18]:
# Initialize the parameters
confThreshold = 0.5  #Confidence threshold-> 낮은 값은 제거
nmsThreshold = 0.4   #Non-maximum suppression threshold-> 거리가 가까우면 같은 class으로 봄
# 모든 이미지를 416 * 416으로 변환
inpWidth = 416       #Width of network's input image
inpHeight = 416      #Height of network's input image


# Load names of classes
classesFile = "./yolo/coco.names" 
classes = None
with open(classesFile, 'rt') as f:
    classes = f.read().rstrip('\n').split('\n')
print(classes)


# yolo는 독자적인 파일을 사용함(옛날형식으로 )
# Give the configuration and weight files for the model and load the network using them.
modelConfiguration = "./yolo/yolov3.cfg"  # network 구조 정보
modelWeights = "./yolo/yolov3.weights"

['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


In [19]:
net = cv2.dnn.readNetFromDarknet(modelConfiguration, modelWeights)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

## make functions

In [20]:
# Get the names of the output layers
def getOutputsNames(net):
    # Get the names of all the layers in the network
    layersNames = net.getLayerNames()
    # Get the names of the output layers, i.e. the layers with unconnected outputs
    return [layersNames[i[0] - 1] for i in net.getUnconnectedOutLayers()]

In [21]:
# Draw the predicted bounding box
def crop_drawPred(dog_dir,classId, conf, left, top, right, bottom):
    # Draw a bounding box.
    cv2.rectangle(frame, (left, top), (right, bottom), (255, 178, 50), 3)
    
    label = '%.2f' % conf
        
    # Get the label for the class name and its confidence
    if classes:
        assert(classId < len(classes))
        label = '%s:%s' % (classes[classId], label)

    #Display the label at the top of the bounding box
    labelSize, baseLine = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
    top = max(top, labelSize[1])
    cv2.rectangle(frame, (left, top - round(1.5*labelSize[1])), (left + round(1.5*labelSize[0]), top + baseLine), (255, 255, 255), cv2.FILLED)
    cv2.putText(frame, label, (left, top), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,0,0), 1)
#     imshow('', frame)  

In [22]:
# Remove the bounding boxes with low confidence using non-maxima suppression
def crop_postprocess(frame, outs,new_dogs_dir,new_dog_dir,idx):
    frameHeight = frame.shape[0]
    frameWidth = frame.shape[1]

    # Scan through all the bounding boxes output from the network and keep only the
    # ones with high confidence scores. Assign the box's class label as the class with the highest score.
    classIds = []
    confidences = []
    boxes = []
    centerX=[]
    centerY=[]

    for out in outs:
        for detection in out:
            scores = detection[5:]
            classId = np.argmax(scores)
            confidence = scores[classId]
            if confidence > confThreshold:
                center_x = int(detection[0] * frameWidth)
                center_y = int(detection[1] * frameHeight)
                width = int(detection[2] * frameWidth)
                height = int(detection[3] * frameHeight)
                left = int(center_x - width / 2)
                top = int(center_y - height / 2)
                classIds.append(classId)
                confidences.append(float(confidence))
                boxes.append([left, top, width, height])
                centerX.append(center_x)
                centerY.append(center_y)

    # Perform non maximum suppression to eliminate redundant overlapping boxes with
    # lower confidences.
#     print(boxes)
    indices = cv2.dnn.NMSBoxes(boxes, confidences, confThreshold, nmsThreshold)
#     print(len(indices))
    for i in indices:
        i = i[0]
        box = boxes[i]
        x = centerX[i]
        y = centerY[i]

        left = box[0]
        top = box[1]
        width = box[2]
        height = box[3]
#         crop_drawPred(dog_dir,classIds[i], confidences[i], left, top, left + width, top + height)        
        crop_image(new_dogs_dir,new_dog_dir,idx,i, classIds[i], confidences[i], left, top, width, height,x, y)
   
        

In [23]:
def crop_image(new_dogs_dir,new_dog_dir,idx,i, classIds, confidences, left, top, width, height,x, y):
#     print(i,classIds, width, height)
    if classIds == 16:
        if width >= height:
            left = int(x - width / 2)
            top = int(y - width / 2)
            right= int(x + width / 2)
            bottom= int(y + width / 2)
        else:
            left = int(x - height / 2)
            top = int(y - height / 2)
            right= int(x + height / 2)
            bottom= int(y + height / 2)

        if left <0:
            margin = np.abs(left)
            left = 0
            top = int(top + margin/2)
            bottom = int(bottom - margin/2)

        if top <0:
            margin = np.abs(top)
            top = 0
            left = int(left + margin/2)
            right = int(right - margin/2)


        crop_img = frame_crop[top:bottom, left:right]
        crop_img = cv2.cvtColor(crop_img, cv2.COLOR_RGB2BGR)
        save_path = f"{new_dogs_dir}/{new_dog_dir}_{idx}_{i}.jpg"
        cv2.imwrite(save_path, crop_img) 
#         imshow("cropped", crop_img)

##  img_path

In [31]:
import os

In [32]:
base_path = '../data/'

In [34]:
old_dir = os.path.join(base_path, '5.merge(stanfold,identification,crawling)_120').replace('\\','/')
old_dir

'../data/5.merge(stanfold,identification,crawling)_120'

In [35]:
new_dir = '../data/6.image crop by YOLO3'
try:
    os.mkdir(new_dir)
except Exception as err:
    print(err)

[WinError 183] 파일이 이미 있으므로 만들 수 없습니다: '../data/6.image crop by YOLO3'


In [38]:
dogs_dir = os.listdir(old_dir)
dogs_dir

['affenpinscher',
 'afghan_hound',
 'airedaleterrier',
 'akita',
 'alaskanmalamute',
 'americancockerspaniel',
 'americaneskimodog',
 'australian_terrier',
 'basenji',
 'bassethound',
 'beagle',
 'beardedcollie',
 'bedlington_terrier',
 'belgiansheepdoggroenendael',
 'belgiansheepdoglaekenois',
 'belgiansheepdogmalinois',
 'belgiansheepdogtervuren',
 'bernese_mountain_dog',
 'bichonfrise',
 'black-and-tan_coonhound',
 'bloodhound',
 'border_collie',
 'border_terrier',
 'borzoi',
 'bostonterrier',
 'boxer',
 'brittanydog',
 'brusselsgriffonr',
 'bulldog',
 'bullterrier',
 'bull_mastiff',
 'cairnterrier',
 'caucasianavtcharka',
 'chesapeake_bay_retriever',
 'chihuahua',
 'chinesecrested',
 'chinesesharpei',
 'chowchow',
 'clumberspaniel',
 'collie',
 'curly-coated_retriever',
 'dachshunds',
 'dalmatian',
 'dandiedinmontterrier',
 'dobermanpinscher',
 'englishcockerspaniel',
 'englishspringerspaniel',
 'english_setter',
 'fieldspaniel',
 'flat-coated_retriever',
 'french_bulldog',
 'germa

In [ ]:
from tqdm import tqdm

for dog_dir in tqdm(dogs_dir):
#     print(dog_dir)
    dir_path = os.path.join(old_dir, dog_dir)
    print(dir_path)
    try:
        new_dog_dir = str(dog_dir).upper()
        new_dogs_dir =os.path.join(new_dir, new_dog_dir).replace('\\','/') 
#         print('new_dogs_dir',new_dogs_dir)
        os.mkdir(new_dogs_dir)
    except Exception as err:
        print(err)
    img_list = os.listdir(dir_path)

    for idx, img_name in enumerate(img_list):
#         print(idx, img_name)
        img_path = os.path.join(dir_path, img_name)
#         print(img_path)
        try:
            img = cv2.imread(img_path, 1)
            frame =cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # 오브젝트가 없는 경우
        except Exception as err:
            print(img_path)
            print(err)
            continue
        frame_crop = frame.copy()
        blob = cv2.dnn.blobFromImage(frame, 1/255, (inpWidth, inpHeight), [0,0,0], 1, crop=False)

        net.setInput(blob)

        outs = net.forward(getOutputsNames(net))

        crop_postprocess(frame, outs, new_dogs_dir,new_dog_dir, idx)
        



  0%|                                                                                          | 0/121 [00:00<?, ?it/s]

../data/5.merge(stanfold,identification,crawling)_120\affenpinscher
[WinError 183] 파일이 이미 있으므로 만들 수 없습니다: '../data/6.image crop by YOLO3/AFFENPINSCHER'
../data/5.merge(stanfold,identification,crawling)_120\affenpinscher\affenpinscher37.jpeg
OpenCV(4.1.1) C:\projects\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'





  1%|▋                                                                             | 1/121 [07:13<14:27:41, 433.84s/it]

../data/5.merge(stanfold,identification,crawling)_120\afghan_hound




  2%|█▎                                                                            | 2/121 [17:08<15:55:57, 481.99s/it]

...

../data/5.merge(stanfold,identification,crawling)_120\basenji




  7%|█████▋                                                                      | 9/121 [1:10:21<15:44:42, 506.10s/it]

../data/5.merge(stanfold,identification,crawling)_120\bassethound




  8%|██████▏                                                                    | 10/121 [1:18:46<15:35:50, 505.86s/it]

../data/5.merge(stanfold,identification,crawling)_120\beagle
